# Provably safe ACL and firewall rule changes

Changing ACLs or firewall rules is one of the riskiest update to the network. Even a small error can block connectivity for a large set of critical services or open up sensitive resources to the world at large. 

This notebook shows how Batfish can update ACLs and firewall rules, which we generally call filters, in a provably correct and safe manner. For a broader view of Batfish's support for filter analysis, check out [this notebook](Analyzing%20ACLs%20and%20Firewall%20Rules.ipynb).
![Analytics](https://ga-beacon.appspot.com/UA-100596389-3/open-source/pybatfish/jupyter_notebooks/provably-safe-filter-changes?pixel&useReferer) 

## Change scenario

We mimic a scenario where we want to permit HTTP traffic (ports 80 and 8080) from one subnet to another. We will implement this by adding a rule to permit this traffic to our filter, and we will then use Batfish to check if the implementation was correct.

In [9]:
# The source and destination prefix for the traffic to allow
src_prefix = "10.10.20.10/26"
dst_prefix = "18.18.18.0/24"
ip_protocol = "tcp"
dst_ports = [80, 8080]

# The name of the filter to change
filter_name = "851034_in_20171004133715"

## Step 1: Intialize the current (pre-change) snapshot

We start by initializing the pre-change snapshot. In our example, the snapshot contains two devices, and we'll change the ACL on `rtr-with-acl`, whose configuration can be seen [here](https://FILL ME/). 

In [14]:
# Import packages and load questions
%run startup.py

# Initialize a network and snapshot
NETWORK_NAME = "filter_analysis_network"
INITIAL_SNAPSHOT_NAME = "initial"
INITIAL_SNAPSHOT_PATH = "networks/example-filters"

bf_set_network(NETWORK_NAME)
bf_init_snapshot(INITIAL_SNAPSHOT_PATH, name=INITIAL_SNAPSHOT_NAME, overwrite=True)

'initial'

## Step 2:  Ensure that the intended traffic is not already permitted

Before we make the change to allow the intended traffic, we verify that that traffic is not already permitted---because if it is, we are done and do not need to do anything. We accomplish this using the `reachfilter` question, which finds flows that match the specified criteria.  

In [12]:
# Check if the intended traffic is already permitted in the initial snapshot
answer = bfq.reachfilter(src=src_prefix,
                         dst=dst_prefix, 
                         dstPorts=dst_ports,
                         #ipProtocols="tcp",
                         filterRegex=filter_name,
                         query="permit").answer(snaphot=INITIAL_SNAPSHOT_NAME).frame()
display_html(answer)

,Node,Filter_Name,Flow,Action,Line_Number,Line_Content,Trace
0,rtr-with-acl,851034_in_20171004133715,rtr-with-acl->[10.10.20.0:0->18.18.18.0:80 proto: ICMP dscp:0 ecn:0 fragOff:0 length:0 state:NEW flags: n/a,PERMIT,67,1320 permit icmp any any echo-reply,"Flow permitted by 'extended ipv4 access-list' named '851034_in_20171004133715', index 67: 1320 permit icmp any any echo-reply"


Since the query above did not find any flows, we know with certainty that no flow within the specified space is already permitted. We can now proceed.

**TODO**
1. Support ipProtocols (that is when this query should return zero results)

## Step C1.1: Implement and load the change

Assume that we implemented the change and stored it in the `CANDIDATE_SNAPSHOT_PATH` (see below). The diff of the change is:

**FILL**

Now we will load the proposed change into Batfish so that we can analyze it. 

In [17]:
CANDIDATE1_SNAPSHOT_NAME = "candidate1"
CANDIDATE1_SNAPSHOT_PATH = "networks/example-filters-candidate1"

bf_init_snapshot(CANDIDATE1_SNAPSHOT_PATH, name=CANDIDATE1_SNAPSHOT_NAME, overwrite=True)

FileNotFoundError: [Errno 2] No such file or directory: 'networks/example-filters-candidate1'

## Step C1.2: Ensure that the intended traffic is now permitted

We run the same query as in Step 2 above, but on the candidate1 snapshot.

In [15]:
# Check if the intended traffic is already permitted in the candidate1 snapshot
answer = bfq.reachfilter(src=src_prefix,
                         dst=dst_prefix, 
                         dstPorts=dst_ports,
                         #ipProtocols="tcp",
                         filterRegex=filter_name,
                         query="permit").answer(snaphot=CANDIDATE1_SNAPSHOT_NAME).frame()
display_html(answer)

NameError: name 'CANDIDATE1_SNAPSHOT_NAME' is not defined

Since we got no results, we can be confident that our candidate change permits *all* traffic that we intended to permit. If there were any flow in the desired space that was not permitted by the change, the query above would have found it.

## Step C1.3: Ensure that no other traffic is impacted

Typically, engineers will stop change validation after checking that the intended traffic has been successfully permitted by the change. However, we must also check that no traffic outside of the intended space has been impacted--that is, our change has not caused collateral damage. 

We can verify that by using a "differential" version of the `reachfilter` question that compares two snapshots. The query below compares the candidate1 and initial snapshots, and is asking Batfish to return any flow outside of the intended traffic that the two snapshots treat differently (i.e., one of them permits and the other rejects, or vice versa). The "outside" part is specified using the `complementHeaderSpace` parameter; in its absence, the search for differently-treated flows is conducted *inside* the specified space. If our query, returns no result, then combined with the result above, we have ensured that the change is completely correct.

In [168]:
# Check if traffic other than the intended traffic has been impacted
answer = bfq.reachfilter(src=src_prefix,
                         dst=dst_prefix, 
                         dstPorts=dst_ports,
                         #ipProtocols="tcp",
                         filterRegex=filter_name,
                         complementHeaderSpace=True).answer(snapshot=CANDIDATE1_SNAPSHOT_NAME, 
                                                            reference_snapshot=INITIAL_SNAPSHOT_NAME).frame()
display_html(answer)

,Node,Filter_Name,HeaderSpace,Flow,KeyPresence,Diff_Action,Base_Action,Delta_Action,Diff_Line_Number,Base_Line_Number,Delta_Line_Number,Diff_Line_Content,Base_Line_Content,Delta_Line_Content,Diff_Trace,Base_Trace,Delta_Trace
0,lvs1-ar01,851034_in,"Including dstIps: 12.125.93.0/26, dstPorts: 80, ipProtocols: TCP, srcIps: 0.0.0.0/0Excluding srcIps: 2.80.3.219Excluding srcIps: 8.89.120.0/22Excluding srcIps: 10.64.90.249Excluding srcIps: 11.36.192.0/19Excluding srcIps: 11.125.64.0/19Excluding srcIps: 27.212.145.150Excluding srcIps: 28.69.250.136Excluding srcIps: 29.228.179.210Excluding srcIps: 54.0.0.0/8Excluding srcIps: 118.0.0.0/8Excluding srcIps: 131.159.53.215Excluding srcIps: 144.49.45.40Excluding srcIps: 163.157.0.0/16Excluding srcIps: 163.255.18.63Excluding srcIps: 166.144.0.0/12Excluding srcIps: 194.181.135.214Excluding srcIps: 196.64.84.239Excluding srcIps: 198.120.0.0/16Excluding srcIps: 198.170.50.0/24Excluding srcIps: 200.45.87.238Excluding srcIps: 202.45.130.141Excluding srcIps: 207.70.46.217Excluding srcIps: 212.26.132.18Excluding srcIps: 213.78.84.0/22Excluding srcIps: 214.220.213.107Excluding srcIps: 218.8.104.0/25Excluding srcIps: 218.8.106.128/25Excluding srcIps: 218.8.107.0/25Excluding srcIps: 218.66.51.0/24Excluding srcIps: 218.66.57.0/24Excluding srcIps: 218.67.46.0/23Excluding srcIps: 218.67.96.0/22Excluding srcIps: 218.67.213.0/24Excluding srcIps: 218.111.16.132Excluding srcIps: 218.246.165.90Excluding srcIps: 219.185.241.117Excluding srcIps: 224.0.0.0/3Excluding ipProtocols: TCP, tcpFlags: [ack=True, rst=True]Excluding dstIps: 12.125.93.0/27, dstPorts: [80, 443], ipProtocols: TCP, srcIps: anyIncluding dstIps: 12.125.93.0/26, dstPorts: 443, ipProtocols: TCP, srcIps: 0.0.0.0/0Excluding srcIps: 2.80.3.219Excluding srcIps: 8.89.120.0/22Excluding srcIps: 10.64.90.249Excluding srcIps: 11.36.192.0/19Excluding srcIps: 11.125.64.0/19Excluding srcIps: 27.212.145.150Excluding srcIps: 28.69.250.136Excluding srcIps: 29.228.179.210Excluding srcIps: 54.0.0.0/8Excluding srcIps: 118.0.0.0/8Excluding srcIps: 131.159.53.215Excluding srcIps: 144.49.45.40Excluding srcIps: 163.157.0.0/16Excluding srcIps: 163.255.18.63Excluding srcIps: 166.144.0.0/12Excluding srcIps: 194.181.135.214Excluding srcIps: 196.64.84.239Excluding srcIps: 198.120.0.0/16Excluding srcIps: 198.170.50.0/24Excluding srcIps: 200.45.87.238Excluding srcIps: 202.45.130.141Excluding srcIps: 207.70.46.217Excluding srcIps: 212.26.132.18Excluding srcIps: 213.78.84.0/22Excluding srcIps: 214.220.213.107Excluding srcIps: 218.8.104.0/25Excluding srcIps: 218.8.106.128/25Excluding srcIps: 218.8.107.0/25Excluding srcIps: 218.66.51.0/24Excluding srcIps: 218.66.57.0/24Excluding srcIps: 218.67.46.0/23Excluding srcIps: 218.67.96.0/22Excluding srcIps: 218.67.213.0/24Excluding srcIps: 218.111.16.132Excluding srcIps: 218.246.165.90Excluding srcIps: 219.185.241.117Excluding srcIps: 224.0.0.0/3Excluding ipProtocols: TCP, tcpFlags: [ack=True, rst=True]Excluding dstIps: 12.125.93.0/27, dstPorts: [80, 443], ipProtocols: TCP, srcIps: any",lvs1-ar01->[0.0.0.0:0->12.125.93.32:80 proto: TCP dscp:0 ecn:0 fragOff:0 length:0 state:NEW flags: n/a,In both,,DENY,PERMIT,: 527,976,449,,19590 deny tcp any any,9032 permit tcp any 12.125.93.10/26 eq www,,"Flow denied by 'extended ipv4 access-list' named '851034_in', index 976: 19590 deny tcp any any","Flow permitted by 'extended ipv4 access-list' named '851034_in', index 449: 9032 permit tcp any 12.125.93.10/26 eq www"


As we see, unfortunately, we do get some results. In particular, we have accidentally allowed traffic that we didn't intend to. The columns of the result show what traffic that is and how it is treated differently. 

The root cause of the problem is that we in our candidate1 change, we ended permitting /25 source prefix instead of the intended /26 prefix. We thus need to fix this.

## Step C2.1: Implement and load the change

Assume that we re-implemented the change and stored it in the `CANDIDATE2_SNAPSHOT_PATH`. We can now load the change for analysis. For reference, the diff view of the change, with respect to the initial snapshot, is: 

**FILL**

In [18]:
CANDIDATE2_SNAPSHOT_NAME = "candidate2"
CANDIDATE2_SNAPSHOT_PATH = "networks/example-filters-candidate2"

bf_init_snapshot(CANDIDATE2_SNAPSHOT_PATH, name=CANDIDATE2_SNAPSHOT_NAME, overwrite=True)

FileNotFoundError: [Errno 2] No such file or directory: 'networks/example-filters-candidate2'

## Step C2.2: Ensure that the intended traffic is now permitted

We run the same query as in Step 2 above, but on the candidate2 snapshot.

In [15]:
# Check if the intended traffic is already permitted in the candidate2 snapshot
answer = bfq.reachfilter(src=src_prefix,
                         dst=dst_prefix, 
                         dstPorts=dst_ports,
                         #ipProtocols="tcp",
                         filterRegex=filter_name,
                         query="permit").answer(snaphot=CANDIDATE2_SNAPSHOT_NAME).frame()
display_html(answer)

NameError: name 'CANDIDATE1_SNAPSHOT_NAME' is not defined

As before, we got no results, which means that we correctly permitted all intended traffic. 

## Step C2.3: Ensure that no other traffic is impacted

Now, lets check again that no other traffic is impacted.

In [19]:
# Check if traffic other than the intended traffic has been impacted
answer = bfq.reachfilter(src=src_prefix,
                         dst=dst_prefix, 
                         dstPorts=dst_ports,
                         #ipProtocols="tcp",
                         filterRegex=filter_name,
                         complementHeaderSpace=True).answer(snapshot=CANDIDATE2_SNAPSHOT_NAME, 
                                                            reference_snapshot=INITIAL_SNAPSHOT_NAME).frame()
display_html(answer)

BatfishException: Coordinator returned failure: Snapshot 'candidate2' does not exist

This time, we got no results. Thus, this change is completely correct: It allows all traffic that we meant to allow and has no impact of any other traffic. 

## Conclusions

In this notebook, we showed how you can use Batfish to validate that a change to a complex ACL/filter is correct and only permits or denies the desired set of traffic/flows. The following 3 steps provide that guarantee:

1. Check that the intended traffic does not already match the desired action (permit or deny) in the existing configuration
2. Check that the intended traffic is matched with the desired action in the changed configuration
3. Check that nothing but the intended traffic is impacted by the change

This simple 3 step process gives you the assurance needed to be able to quickly and safely make changes to ACLs/filters in your network. For general filter analysis using Batfish, see [this notebook](Analyzing%20ACLs%20and%20Firewall%20Rules.ipynb).

***
### Get involved with the Batfish community

To get involved and learn more, join our community on [Slack](https://join.slack.com/t/batfish-org/shared_invite/enQtMzA0Nzg2OTAzNzQ1LTUxOTJlY2YyNTVlNGQ3MTJkOTIwZTU2YjY3YzRjZWFiYzE4ODE5ODZiNjA4NGI5NTJhZmU2ZTllOTMwZDhjMzA) and [Github](https://github.com/batfish/batfish). 